In [2]:
import sqlite3
import pandas as pd

# Dashboard User Function
Functions to add, delete, and update users.

In [3]:
def insert_user(db_path, username, first, last, email, has_password=0):
    """
    Inserts a user into the user database. Will throw an error is the user
    already exists.

    Args:
        db_path: path to the user database
        username: username of the user
        first: users first name
        last: users last name
        email: user email including address
        has_password: indicates if they have a password in the users table
            should be 0 in almost every situation.
        
    Returns:
        None: updates user database
    """
    conn = sqlite3.connect(db_path)

    new_user = pd.DataFrame(
        [[username, first, last, email, has_password]],
        columns=["username", "first_name", "last_name", "email", "has_password"],
    )
    new_user.to_sql("usernames", conn, if_exists="append", index=False)

    conn.commit()
    conn.close()

    return "User inserted"

In [4]:
def delete_user(db_path, username):
    """
    Deletes a user from the user database. 

    Args:
        db_path: path to the user database
        username: username of the user
        
    Returns:
        None: updates user database
    """
    conn = sqlite3.connect(db_path)

    query_user = """DELETE FROM user WHERE username=?;"""
    query_usernames = """DELETE FROM usernames WHERE username=?;"""

    c = conn.cursor()
    c.execute(query_user, (username,))
    c.execute(query_usernames, (username,))

    conn.commit()
    conn.close()

    return "User deleted"

In [5]:
def update_from_csv(user_db, csv_path):
    """
    Updates usernames table in the users database using the usernames csv

    Args:
        db_path: path to the user database
        csv_path: path of the csv to use to update the database
        
    Returns:
        None: updates user database
    """
    usernames = pd.read_csv(csv_path)

    conn = sqlite3.connect(user_db)
    c = conn.cursor()

    usernames.to_sql("temp", conn, if_exists="append", index=False)
    c.execute(
        f"""
            INSERT INTO usernames (username, first_name, last_name, email, has_password)
            SELECT username, first_name, last_name, email, has_password FROM temp t
            WHERE NOT EXISTS
                (SELECT * from usernames f
                WHERE f.username = t.username);
            """
    )
    conn.commit()
    conn.close()
    return "Users updated"

In [6]:
def create_from_csv(db_path, csv_path):
    """
    Creates a users database for the dashboard from
    a provided csv file

    Args:
        db_path(str): path to the database - if just the name
            it will be created in this folder
        csv_path(str): path to the csv
            that contains the csv of users
            Columns of username, first_name, last_name, email, has_password
    """
    usernames = pd.read_csv(csv_path)
    conn = sqlite3.connect(db_path)
    c = conn.cursor()

    q = """CREATE TABLE user (
            id INTEGER PRIMARY KEY,
            username TEXT NOT NULL UNIQUE,
            email TEXT NOT NULL UNIQUE,
            password NOT NULL
            );
            """

    c.execute(q)
    conn.commit()

    q = """CREATE TABLE usernames (
            username TEXT NOT NULL UNIQUE PRIMARY KEY,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT NOT NULL UNIQUE,
            has_password INT NOT NULL
            );
        """
    c.execute(q)

    usernames.to_sql("usernames", conn, if_exists="append", index=False)
    conn.commit()
    conn.close()

    return "User database created"

In [7]:
def reset_password(db_path, username):
    """
    Resets a user's password by deleting the user table and
    setting has_password to 0 in the usernames table. 

    Args:
        db_path: path to the user database
        username: username of the user
        
    Returns:
        None: updates user database
    """
    conn = sqlite3.connect(db_path)

    delete_password_indication = """UPDATE usernames SET has_password=0 WHERE username=?"""
    delete_user_login = """DELETE FROM user WHERE username=?;"""

    c = conn.cursor()
    c.execute(delete_user_login, (username,))
    c.execute(delete_password_indication, (username,))

    conn.commit()
    conn.close()
    
    return "Password ready to be reset."

In [8]:
delete_user("V:/Databases/users.db", "mkehoe")

'User deleted'

In [9]:
insert_user("V:/Databases/users.db", "mgirard", "Maryellen", "Girard", "mgirard@pace-ri.org", has_password=0)

'User inserted'